In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import keplergl
#from keplergl import KeplerGL
import matplotlib as plt
import plotly.express as px
import h3
import shapely
from geopandas import GeoDataFrame,GeoSeries
import folium
import matplotlib
%matplotlib inline

# Automated Speed Enforcement

In [10]:
speed_enforcement = gpd.read_file('Automated Speed Enforcement Locations.geojson')

In [11]:
speed_enforcement.head()

,_id,Location_Code,Ward,Status,location,FID,geometry
0,1,A298,1 - Etobicoke North,Active,Thistle Down Blvd. East of Albion Rd.,1,MULTIPOINT (-79.56022 43.73667)
1,2,A318,1 - Etobicoke North,Active,Kipling Ave. North of Coolhurst Dr.,2,MULTIPOINT (-79.56942 43.71741)
2,3,A319,1 - Etobicoke North,Active,Mount Olive Dr. North of Dorward Dr.,3,MULTIPOINT (-79.59493 43.74885)
3,4,A252,2 - Etobicoke Centre,Active,Bloor St. W. Near 4034 Bloor St. W.,4,MULTIPOINT (-79.55458 43.63745)
4,5,A253,2 - Etobicoke Centre,Active,Allanhurst Dr. Near Mulham Place,5,MULTIPOINT (-79.51767 43.68003)


In [12]:
#convert column names to lowercase
speed_enforcement.columns = [s.strip().lower() for s in speed_enforcement.columns]

In [13]:
#set location code as index
speed_enforcement = speed_enforcement.set_index('location_code')

In [14]:
speed_enforcement = speed_enforcement.drop(columns=['_id','ward','status','location','fid'])

In [15]:
speed_enforcement.head()

,geometry
location_code,
A298,MULTIPOINT (-79.56022 43.73667)
A318,MULTIPOINT (-79.56942 43.71741)
A319,MULTIPOINT (-79.59493 43.74885)
A252,MULTIPOINT (-79.55458 43.63745)
A253,MULTIPOINT (-79.51767 43.68003)


In [16]:
#converting multipoing geometry to single point geometry
speed_enforcement = speed_enforcement.explode()

C:\Users\arara\AppData\Local\Temp\ipykernel_80100\2596447337.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  speed_enforcement = speed_enforcement.explode()


In [19]:
#extract latitude and longitudes from single line geometry
speed_enforcement['latitude'] = speed_enforcement.apply(lambda x: x.geometry.y, axis=1)
speed_enforcement['longitude'] = speed_enforcement.apply(lambda x: x.geometry.x, axis=1)

In [22]:
#combining latitude and longitudes to form coordinates column
speed_enforcement['coordinates'] = speed_enforcement.apply(lambda x: (x.latitude, x.longitude), axis=1)

#create hex cells for bike lanes

#create coordinate column
#pedestrian_crossover_exploded['coordinates'] = pedestrian_crossover_exploded['latitude'].astype('str') + ',' + pedestrian_crossover_exploded['longitude'].astype('str')

#then we define the function to get h3 hex cells from coordinates data
def get_hex_cell(coordinates):
    #a = float(coordinates.split(',')[0])
    #b = float(coordinates.split(',')[1])
    a = coordinates[0]
    b = coordinates[1]
    return h3.latlng_to_cell(a,b,res=10)

#finally, create a new column for hex cells by applying the function on coordinates column
speed_enforcement['hex_cell'] = speed_enforcement['coordinates'].apply(get_hex_cell)

In [25]:
speed_enforcement.head()

,,geometry,latitude,longitude,coordinates,hex_cell
location_code,,,,,,
A298,0,POINT (-79.56022 43.73667),43.736670,-79.560220,"(43.73667, -79.56022)",8a2b9bc1e967fff
A318,0,POINT (-79.56942 43.71741),43.717410,-79.569425,"(43.71741, -79.569425)",8a2b9bc1e25ffff
A319,0,POINT (-79.59493 43.74885),43.748847,-79.594933,"(43.748847, -79.594933)",8a2b9bc131affff
A252,0,POINT (-79.55458 43.63745),43.637452,-79.554583,"(43.637452, -79.554583)",8a2b9bce3437fff
A253,0,POINT (-79.51767 43.68003),43.680032,-79.517675,"(43.680032, -79.517675)",8a2b9bce4c4ffff


In [84]:
speed_enforcement.to_csv('speed_enforcement_h3.csv')

# Traffic Camera List

In [75]:
traffic_cameras = pd.read_csv('Traffic Camera List (CSV).csv')

In [76]:
#converting column headers to lowercase
traffic_cameras.columns = [s.strip().lower().replace(' ','_') for s in traffic_cameras.columns]

In [77]:
traffic_cameras.head()

,camera_number,latitude,longitude,main_road,cross_street,group,traffic_image,north_reference_static_image,east_reference_static_image,south_reference_static_image,west_reference_static_image
0,Camera8000,43.649450,-79.371464,ABELL ST,BABY POINT RD,Arterial,http://opendata.toronto.ca/transportation/tmc...,,,,NaN
1,Camera8001,43.643120,-79.381386,YORK ST,BREMNER BLVD / RAPTORS WAY,Arterial,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...
2,Camera8002,43.642220,-79.384068,LOWER SIMCOE ST,BREMNER BLVD,Arterial,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...
3,Camera8003,43.649461,-79.371267,JARVIS ST,FRONT ST E / LOWER JARVIS ST,Arterial,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...
4,Camera8004,43.645153,-79.382574,UNIVERSITY AVE,FRONT ST W / YORK ST,Arterial,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...


In [78]:
#set index to camera number
traffic_cameras = traffic_cameras.set_index('camera_number')

In [79]:
#dropping unnecessary columns
traffic_cameras = traffic_cameras.drop(traffic_cameras.columns[3:],axis=1)

In [80]:
traffic_cameras

,latitude,longitude,main_road
camera_number,,,
Camera8000,43.649450,-79.371464,ABELL ST
Camera8001,43.643120,-79.381386,YORK ST
Camera8002,43.642220,-79.384068,LOWER SIMCOE ST
Camera8003,43.649461,-79.371267,JARVIS ST
Camera8004,43.645153,-79.382574,UNIVERSITY AVE
...,...,...,...
Camera9403,43.711222,-79.442755,W R ALLEN RD
Camera9404,43.716458,-79.444790,W R ALLEN RD
Camera9405,43.728143,-79.448634,W R ALLEN RD


In [81]:
#combining latitude and longitudes to form coordinates column
traffic_cameras['coordinates'] = traffic_cameras.apply(lambda x: (x.latitude, x.longitude), axis=1)

#create hex cells for bike lanes

#create coordinate column
#pedestrian_crossover_exploded['coordinates'] = pedestrian_crossover_exploded['latitude'].astype('str') + ',' + pedestrian_crossover_exploded['longitude'].astype('str')

#then we define the function to get h3 hex cells from coordinates data
def get_hex_cell(coordinates):
    #a = float(coordinates.split(',')[0])
    #b = float(coordinates.split(',')[1])
    a = coordinates[0]
    b = coordinates[1]
    return h3.latlng_to_cell(a,b,res=10)

#finally, create a new column for hex cells by applying the function on coordinates column
traffic_cameras['hex_cell'] = traffic_cameras['coordinates'].apply(get_hex_cell)

In [82]:
traffic_cameras.head()

,latitude,longitude,main_road,coordinates,hex_cell
camera_number,,,,,
Camera8000,43.649450,-79.371464,ABELL ST,"(43.64945, -79.371464)",8a2b9bc4686ffff
Camera8001,43.643120,-79.381386,YORK ST,"(43.64312, -79.381386)",8a2b9bc4604ffff
Camera8002,43.642220,-79.384068,LOWER SIMCOE ST,"(43.64222, -79.384068)",8a2b9bc463b7fff
Camera8003,43.649461,-79.371267,JARVIS ST,"(43.649461, -79.371267)",8a2b9bc4686ffff
Camera8004,43.645153,-79.382574,UNIVERSITY AVE,"(43.645153, -79.382574)",8a2b9bc46077fff


In [83]:
traffic_cameras.to_csv('traffic_cameras_h3.csv')

Note: These two datasets can be combined into an "Automated Traffic Control" Dataset.